Exercise 1

In [ ]:
## c). LOOCV

Exercise 2

In [ ]:
## a). low bias and low variance

Exercise 3

In [ ]:
## b). low bias and high variance

Exercise 4

In [ ]:
## Regularization are ways to fix or calibrate machine learning models tominimize the adjusting loss function and 
# to prevent overfitting and underfitting.

Exercise 5

In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
## Removing the observations with N/As
heart = heart.dropna()

In [4]:
from sk.learn.metrics import f1_score

X = heart.drop(columns = ['TenYearCHD'], axis = 1)
Y = heart['TenYearCHD']
             
##Defining the folds
kf = KFold(n_splits = 5, shuffle = True)

##Defining the lists to store results
f1_score1 = list()
f1_score2 = list()

for train_idx, val_idx in kf.split(X):
    
    ##Splitting the data
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[val_idx]
    
    ##Model 1
    
    X1 = X_train[['age','currentSmoker','totChol','sysBP','diaBP', 'BMI','heartRate','glucose']]
    X_val_1 = X_val[['age','currentSmoker','totChol','sysBP','diaBP', 'BMI','heartRate','glucose']]
    
    ##Building the linear model
    md1 = LinearRegression().fit(X1, Y_train)
    
    ##Predicting ton the validation set
    pred1 = md1.predict(X_val_1)
    
    ##Storing the f1_score
    f1_score1
    
    ##Model 2
    
    X2 = X_train[['age','currentSmoker','totChol','BMI','heartRate','glucose']]
    X_val_2 = X_val[['age','currentSmoker','totChol','BMI','heartRate','glucose']]
    
    ##Building the linear model
    md2 = LinearRegression().fit(X2, Y_train)
    
    ##Predicting ton the validation set
    pred2 = md2.predict(X_val_2)
    
    ##Storing the f1_score
    f1_score2

In [ ]:
## Changing likelihoods to labels - 25%


In [5]:
## Computing the f1_score
f1_score1_avg = np.mean(f1_score1)
f1_score2_avg = np.mean(f1_score2)

print('The f1_score of model 1 is', f1_score1_avg)
print('The f1_score of model 2 is', f1_score2_avg)

The mse of model 1 is 0.11826553231827086
The mse of model 2 is 0.12027669499003565
